Code to compare Bs inb KTAO agsinst HRApop v1.0 kidney data

## Install and import libraries

In [16]:
%pip install pandas requests

import pandas as pd
import requests
from io import StringIO

Note: you may need to restart the kernel to use updated packages.


## Load data

In [17]:
df_ktao_left_kidney = pd.read_csv('data/Left_kidney_biomarker_mapping.csv')
df_ktao_left_kidney

,biomarker_label,organ,biomarker_id,ensembl_id
0,B2M,Left kidney,HGNC:914,ENSG00000273686
1,CST3,Left kidney,HGNC:2475,ENSG00000101439
2,IGFBP7,Left kidney,HGNC:5476,ENSG00000163453
3,IL18,Left kidney,HGNC:5986,ENSG00000150782
4,ITGAM,Left kidney,HGNC:6149,ENSG00000169896
5,LCN2,Left kidney,HGNC:6526,ENSG00000148346
6,NRXN1,Left kidney,HGNC:8008,ENSG00000179915
7,S100A6,Left kidney,HGNC:10496,ENSG00000197956
8,SERPINE2,Left kidney,HGNC:8951,ENSG00000135919
9,SPP1,Left kidney,HGNC:11255,ENSG00000118785


In [18]:
df_ktao_right_kidney = pd.read_csv('data/Right_kidney_biomarker_mapping.csv')
df_ktao_right_kidney

,biomarker_label,organ,biomarker_id,ensembl_id
0,B2M,Right kidney,HGNC:914,ENSG00000273686
1,CST3,Right kidney,HGNC:2475,ENSG00000101439
2,FABP1,Right kidney,HGNC:3555,ENSG00000163586
3,HAVCR1,Right kidney,HGNC:17866,ENSG00000113249
4,IGFBP7,Right kidney,HGNC:5476,ENSG00000163453
5,IL18,Right kidney,HGNC:5986,ENSG00000150782
6,ITGAM,Right kidney,HGNC:6149,ENSG00000169896
7,NRXN1,Right kidney,HGNC:8008,ENSG00000179915
8,S100A6,Right kidney,HGNC:10496,ENSG00000197956
9,SERPINE2,Right kidney,HGNC:8951,ENSG00000135919


In [22]:
# merge both dfs
df_ktao_merged = pd.concat([df_ktao_right_kidney, df_ktao_left_kidney])
df_ktao_merged

,biomarker_label,organ,biomarker_id,ensembl_id
0,B2M,Right kidney,HGNC:914,ENSG00000273686
1,CST3,Right kidney,HGNC:2475,ENSG00000101439
2,FABP1,Right kidney,HGNC:3555,ENSG00000163586
3,HAVCR1,Right kidney,HGNC:17866,ENSG00000113249
4,IGFBP7,Right kidney,HGNC:5476,ENSG00000163453
5,IL18,Right kidney,HGNC:5986,ENSG00000150782
6,ITGAM,Right kidney,HGNC:6149,ENSG00000169896
7,NRXN1,Right kidney,HGNC:8008,ENSG00000179915
8,S100A6,Right kidney,HGNC:10496,ENSG00000197956
9,SERPINE2,Right kidney,HGNC:8951,ENSG00000135919


In [19]:
#for HRApop v1.0
query_string = '''
#+ summary: Cell Summaries with Biomarker information by Dataset
#+ description: Computes the cell summaries plus biomarkers and their mean gene expression for each organ by dataset.

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX ccf: <http://purl.org/ccf/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>
PREFIX FMA: <http://purl.org/sig/ont/fma/fma>
PREFIX HRA: <https://purl.humanatlas.io/collection/hra-api>
PREFIX HRApop: <https://purl.humanatlas.io/graph/hra-pop>

SELECT DISTINCT ?organ_id ?organ ?dataset ?sex ?tool ?modality ?cell_id ?cell_label ?cell_count ?cell_percentage
  ?biomarker_id ?biomarker_label ?mean_gene_expr_value
FROM HRA:
FROM HRApop:
WHERE {
  {
    ?sample ccf:has_registration_location ?rui_location .
    ?sample ccf:generates_dataset ?dataset .
  } UNION {
    ?block ccf:subdivided_into_sections ?sample .
    ?block ccf:has_registration_location ?rui_location .
    ?sample ccf:generates_dataset ?dataset .
  }

  ?dataset ccf:has_cell_summary [ 
    ccf:cell_annotation_method ?tool ;
    ccf:modality ?modality ;
    ccf:has_cell_summary_row [
      ccf:cell_id ?cell_id ;
      ccf:cell_label ?raw_cell_label ;
      ccf:cell_count ?cell_count ;
      ccf:percentage_of_total ?cell_percentage ;
      ccf:gene_expr [
        ccf:gene_label ?biomarker_label ;
        ccf:gene_id ?biomarker_id ;
        ccf:mean_gene_expr_value ?mean_gene_expr_value ;
      ]
    ]
  ] .

  [] a ccf:SpatialPlacement ;
    ccf:placement_relative_to ?refOrgan ;
    ccf:placement_for ?rui_location .

  ?refOrgan owl:sameAs* [
    ccf:representation_of ?organ_id ;
    ccf:organ_owner_sex ?sex ;
    skos:prefLabel ?organ
  ] .

  ?organ_id rdfs:label ?organ_label .

  OPTIONAL { ?cell_id rdfs:label ?rdfs_cell_label . }
  BIND(COALESCE(?rdfs_cell_label, ?raw_cell_label) as ?cell_label)
}
ORDER BY ?sex ?tool ?dataset DESC(?cell_count)
'''

query

'\n#+ summary: Cell Summaries with Biomarker information by Dataset\n#+ description: Computes the cell summaries plus biomarkers and their mean gene expression for each organ by dataset.\n\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX ccf: <http://purl.org/ccf/>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>\nPREFIX FMA: <http://purl.org/sig/ont/fma/fma>\nPREFIX HRA: <https://purl.humanatlas.io/collection/hra-api>\nPREFIX HRApop: <https://purl.humanatlas.io/graph/hra-pop>\n\nSELECT DISTINCT ?organ_id ?organ ?dataset ?sex ?tool ?modality ?cell_id ?cell_label ?cell_count ?cell_percentage\n  ?biomarker_id ?biomarker_label ?mean_gene_expr_value\nFROM HRA:\nFROM HRApop:\nWHERE {\n  {\n    ?sample ccf:has_registration_location ?rui_location .\n    ?sample ccf:generates_dataset ?dataset .\n  } UNION {\n    ?block ccf:subd

In [20]:
# define endpoint
url = "https://sparql.humanatlas.io/blazegraph/namespace/kb/sparql"

# define parameters
params = {
    "query": query_string,
}

# set header
headers = {
    "Accept": "text/csv"
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# convert text to file-like object
csv_data = StringIO(response.text)

# concert to DataFrame
df = pd.read_csv(csv_data)
df.head()

,organ_id,organ,dataset,sex,tool,modality,cell_id,cell_label,cell_count,cell_percentage,biomarker_id,biomarker_label,mean_gene_expr_value
0,http://purl.obolibrary.org/obo/UBERON_0000948,heart,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002129,Atrial Cardiomyocyte,430,0.402999,HGNC:12403,TTN,2.725458
1,http://purl.obolibrary.org/obo/UBERON_0000948,heart,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002129,Atrial Cardiomyocyte,430,0.402999,HGNC:2928,DMD,2.114956
2,http://purl.obolibrary.org/obo/UBERON_0000948,heart,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002129,Atrial Cardiomyocyte,430,0.402999,HGNC:11068,SLC8A1,1.828498
3,http://purl.obolibrary.org/obo/UBERON_0000948,heart,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002129,Atrial Cardiomyocyte,430,0.402999,HGNC:7551,MYBPC3,1.461267
4,http://purl.obolibrary.org/obo/UBERON_0000948,heart,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002129,Atrial Cardiomyocyte,430,0.402999,HGNC:24098,SORBS2,1.654607


In [21]:
# keep only kidney
df_hra_pop_filtered = df[df['organ'].str.contains('kidney')]
df_hra_pop_filtered

,organ_id,organ,dataset,sex,tool,modality,cell_id,cell_label,cell_count,cell_percentage,biomarker_id,biomarker_label,mean_gene_expr_value
2170,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030012,kidney loop of Henle short descending thin lim...,6383,0.568692,HGNC:28685,OCIAD2,1.806449
2171,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030012,kidney loop of Henle short descending thin lim...,6383,0.568692,HGNC:24930,TMEM176A,1.465505
2172,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030012,kidney loop of Henle short descending thin lim...,6383,0.568692,HGNC:6541,LDHB,3.025730
2173,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030012,kidney loop of Henle short descending thin lim...,6383,0.568692,HGNC:843,ATP5MC3,3.073322
2174,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030012,kidney loop of Henle short descending thin lim...,6383,0.568692,HGNC:7061,MGST1,2.340141
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131245,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000875,non-classical monocyte,2,0.000088,HGNC:38043,SEPTIN7P7,0.125000
131246,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000875,non-classical monocyte,2,0.000088,HGNC:3999,FTL,1.000000
131247,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000875,non-classical monocyte,2,0.000088,HGNC:29720,MRPL50P2,0.250000
131248,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000875,non-classical monocyte,2,0.000088,ensembl:ENSG00000277662.1,ENSG00000277662.1,0.166666


## Check for matches between KTAO and HRApop v1.0

In [26]:
df_merged = pd.merge(df_hra_pop_filtered, df_ktao_merged, on=["biomarker_id","biomarker_label","organ"], how="inner")
df_merged

,organ_id,organ,dataset,sex,tool,modality,cell_id,cell_label,cell_count,cell_percentage,biomarker_id,biomarker_label,mean_gene_expr_value,ensembl_id
0,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002201,renal beta-intercalated cell,4,0.000356,HGNC:10496,S100A6,3.694490,ENSG00000197956
1,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://api.cellxgene.cziscience.com/dp/v1/col...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000990,conventional dendritic cell,4,0.000356,HGNC:914,B2M,3.748658,ENSG00000273686
2,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000084,T cell,15,0.001875,HGNC:914,B2M,1.333333,ENSG00000273686
3,http://purl.obolibrary.org/obo/UBERON_0004538,Left kidney,https://entity.api.hubmapconsortium.org/entiti...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030016,epithelial cell of early distal convoluted tubule,369,0.023338,HGNC:11255,SPP1,0.148600,ENSG00000118785
4,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_1000597,papillary tips cell,942,0.134629,HGNC:10496,S100A6,8.969618,ENSG00000197956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_1001005,glomerular capillary endothelial cell,156,0.006880,HGNC:914,B2M,1.435898,ENSG00000273686
770,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030018,kidney connecting tubule principal cell,65,0.002866,HGNC:5476,IGFBP7,0.769231,ENSG00000163453
771,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000890,alternatively activated macrophage,58,0.002558,HGNC:914,B2M,0.775862,ENSG00000273686
772,http://purl.obolibrary.org/obo/UBERON_0004539,Right kidney,https://entity.api.hubmapconsortium.org/entiti...,Male,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_4030017,epithelial cell of late distal convoluted tubule,35,0.001543,HGNC:5476,IGFBP7,0.814286,ENSG00000163453
